## Install segmentation_models via [SO](https://stackoverflow.com/questions/75433717/module-keras-utils-generic-utils-has-no-attribute-get-custom-objects-when-im).

## Code credits: DigitalSreeni [Github](https://github.com/bnsreenu/python_for_microscopists/blob/master/177_semantic_segmentation_made_easy_using_segm_models.py).

In [ ]:
!pip install -U segmentation-models

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

In [ ]:
import tensorflow as tf
import glob
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

## Define the backbone and get the preprocessing function

In [ ]:
BACKBONE = 'vgg19'
preprocess_input = sm.get_preprocessing(BACKBONE)

## Load the files of the minimal dataset

In [ ]:
max_hamming_distance_threshold = 20
hamming_mode = "train"
include_file_name = f"/kaggle/input/cityscapes-processed/data/dhash-cityscapes-results/{hamming_mode}_unique_dhash_{max_hamming_distance_threshold}.txt"

In [ ]:
filtered_dataset_filenames = []

with open(include_file_name, "r") as f:
    lines = f.readlines()
    for line in lines:
        file_name = line.strip()
        filtered_dataset_filenames.append(file_name)

In [ ]:
len(filtered_dataset_filenames)

In [ ]:
filtered_dataset_filenames[:10]

## Dataset creation

In [ ]:
import glob
from tqdm import tqdm

train_images = []
all_images_path = "/kaggle/input/cityscapes-processed/data/processed/train/image"

for dir_path in tqdm(glob.glob(all_images_path)):
    for img_path in tqdm(glob.glob(os.path.join(dir_path, "*.jpg"))):
        # Check for filtered filename in train section
        raw_jpg_path = img_path.split("/")[-1]
        if raw_jpg_path in filtered_dataset_filenames:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            train_images.append(img)

train_images = np.array(train_images, dtype='float32')

train_masks = []
all_masks_path = "/kaggle/input/cityscapes-processed/data/processed/train/binary_road_mask"
for dir_path in tqdm(glob.glob(all_masks_path)):
    for mask_path in tqdm(glob.glob(os.path.join(dir_path, "*.jpg"))):
        raw_jpg_path_mask = mask_path.split("/")[-1]
        # Check for filtered filename in train section
        if raw_jpg_path_mask in filtered_dataset_filenames:
            mask = cv2.imread(mask_path, 0)
            train_masks.append(mask)
        
train_masks = np.array(train_masks,  dtype='float32')

In [ ]:
import glob

val_images = []
all_val_images_path = "/kaggle/input/cityscapes-processed/data/processed/val/image"

for dir_path in tqdm(glob.glob(all_val_images_path)):
    for img_path in glob.glob(os.path.join(dir_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        val_images.append(img)

val_images = np.array(val_images,  dtype='float32')

val_masks = []
all_val_masks_path = "/kaggle/input/cityscapes-processed/data/processed/val/binary_road_mask"
for dir_path in tqdm(glob.glob(all_val_masks_path)):
    for mask_path in tqdm(glob.glob(os.path.join(dir_path, "*.jpg"))):
        mask = cv2.imread(mask_path, 0)
        val_masks.append(mask)
        
val_masks = np.array(val_masks,  dtype='float32')

## Data augmentation strategies
- RandomBrightness
- RandomContrast

In [ ]:
data_augmentation = keras.Sequential([
  keras.layers.RandomBrightness(0.75),
  keras.layers.RandomContrast(0.25),
])

In [ ]:
augmented_train_images = []

for img in tqdm(train_images):
    aug_img = data_augmentation(img)
    assert img.shape == aug_img.shape
    augmented_train_images.append(aug_img)
    
augmented_train_images = np.array(augmented_train_images)

In [ ]:
train_images = augmented_train_images

In [ ]:
X = train_images
Y = train_masks
Y = np.expand_dims(Y, axis = 3)

In [ ]:
X_val = val_images
Y_val = val_masks
Y_val = np.expand_dims(Y_val, axis = 3)

In [ ]:
x_train = X
y_train = Y
# preprocess input
x_train = preprocess_input(x_train)


In [ ]:
x_val = X_val
y_val = Y_val
# preprocess input
x_val = preprocess_input(x_val)


## Define the model

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze = False)

In [ ]:
model.compile(optimizer='adam', loss=sm.losses.CategoricalFocalLoss(), metrics=[sm.metrics.IOUScore(threshold = 0.5), sm.metrics.FScore(threshold=0.5)])

## Simple training run

In [ ]:
history=model.fit(x_train, 
          y_train,
          batch_size=64, 
          epochs=250,
          verbose=1,
          validation_data=(x_val, y_val))

## Plotting metrics

In [ ]:
iou = history.history['iou_score']
iou_val = history.history['val_iou_score']
epochs = range(1, len(iou) + 1)
plt.plot(epochs, iou, 'y', label='Training IOU score')
plt.plot(epochs, iou_val, 'r', label='Validation IOU score')
plt.title(f'Training and validation scores for {BACKBONE}')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.legend()
plt.show()

## Save model

In [ ]:
model.save(f'unet_{BACKBONE}_dhash_train_{max_hamming_distance_threshold}.keras')

## Check ```nvidia-smi``` output

In [ ]:
!nvidia-smi

In [ ]:
!pip install numba 

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()